In [ ]:
sc

In [ ]:
spark

In [ ]:
from graphframes import *
import networkx as nx
import matplotlib.pyplot as plt
import time
import json
import pandas as pd
import re
from datetime import datetime
import subprocess
global start
from pyspark.sql import SparkSession, Row
import os


In [ ]:
stop_word_file = 'stop_word.txt'

# Đọc file và lưu các từ dừng vào một list
with open(stop_word_file, 'r', encoding='utf-8') as file:
    stop_words = [line.strip() for line in file]

letters = [
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
    'a', 'á', 'à', 'ả', 'ã', 'ạ', 'ă', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ', 'â', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ',
    'b', 'c', 'd', 'đ', 'e', 'é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ê', 'ế', 'ề', 'ể', 'ễ', 'ệ',
    'f', 'g', 'h', 'i', 'í', 'ì', 'ỉ', 'ĩ', 'ị', 'j', 'k', 'l', 'm', 'n', 'o', 'ó', 'ò', 'ỏ', 'õ', 'ọ',
    'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ', 'ớ', 'ờ', 'ở', 'ỡ', 'ợ', 'p', 'q', 'r', 's', 't', 'u',
    'ú', 'ù', 'ủ', 'ũ', 'ụ', 'ư', 'ứ', 'ừ', 'ử', 'ữ', 'ự', 'v', 'w', 'x', 'y', 'ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ', 'z'
]

newdata = "/GraphX/New"
olddata = "/GraphX/Old"

In [ ]:


#### đay là hàm kiểm tra số id cao nhất của cây bây giờ là bao nhiêu 
# def maxid(vertices):
#     return vertices.groupBy().max("id").collect()[0]['max(id)']
def max_id(vertices):
    return len(vertices)
###################
####################
#day la ham se ktra lan luot cac tu nho trong cac cay
#stt la bien dua vao de biet chung ta se dung vertices so bao nhieu

def check_letter(src,word,dsts,link):
   # vertices = globals()['vertices'+ str(stt)]
    if len(word)==1:
        row = vertices[letters.index(word)]
        dsts = eval(row[1][1]) # dung eval() de tra ve list
        links = eval(row[1][2])
        if link and link not in links:
            links.append(link)
        ids = row[0]
        new_row = (ids, (row[1][0],str(dsts),str(links)))
        del vertices[ids]
        vertices.insert(ids,new_row)
       
        return ids,dsts        
    elif dsts:
        check = dsts
        for i,value in check:
            row = vertices[i]
            if row[1][0] == word:
                dsts = eval(row[1][1])
                links = eval(row[1][2])
                if link and link not in links:
                    links.append(link)
                ids = row[0]
                new_row = (ids, (row[1][0],str(dsts),str(links)))
                del vertices[ids]
                vertices.insert(ids,new_row)
               
                return ids,dsts
        # update cot truoc
        ids = max_id(vertices)
        row = vertices[src]
        dsts = eval(row[1][1]) 
        dsts.append((ids,word))
        new_row = (row[0], (row[1][0],str(dsts),row[1][2]))
        del vertices[src]
        vertices.insert(src,new_row)
        if link:
            links =[link]
        else:
            links =[]
        #them moi vao cot hien tai
        new_row = (ids, (word,'[]',str(links)))
        vertices.insert(ids,new_row)
       
        return ids,None   
    else:
        # update cot truoc
        ids = max_id(vertices)
        row = vertices[src]
        dsts = eval(row[1][1])
        dsts.append((ids,word))
        new_row = (row[0],(row[1][0],str(dsts),row[1][2]))
        del vertices[src]
        vertices.insert(src,new_row)
        if link:
            links =[link]
        else:
            links =[]
        #them moi vao cot hien tai
        new_row = (ids,(word,'[]',str(links)))
        vertices.insert(ids,new_row)
 
        return ids,None
# day la   ham se kiem tra lan luot ktra tu duoc dua vao trong cacs cay
def check_word(word,link):
    length = len(word)
    dsts = 1221 # con so ngau nhien
    src = 0
    for i in range(1,length):
        char = word[:i]
        src, dsts = check_letter(src,char,dsts,None)
    src, dsts = check_letter(src,word,dsts,link)

def saveTreeData(name, rdd):
    # Lấy ngày giờ hiện tại
    now = datetime.now()

    # Định dạng ngày giờ theo định dạng 'YYYY-MM-DD'
    formatted_time = now.strftime("%Y-%m-%d")

    # Đường dẫn HDFS để lưu đỉnh và cạnh
    link = "/save"
    hdfs_path = link + '/' + str(formatted_time) + "/" + name

    # Câu lệnh HDFS để kiểm tra xem thư mục đã tồn tại chưa
    hdfs_command_check = 'hdfs dfs -test -d ' + hdfs_path

    # Sử dụng subprocess để kiểm tra sự tồn tại của thư mục
    result_check = subprocess.run(hdfs_command_check, shell=True, capture_output=True, text=True)

    # Nếu thư mục tồn tại (lệnh `hdfs dfs -test -d` trả về mã 0), thì xóa thư mục
    if result_check.returncode == 0:
        hdfs_command_delete = 'hdfs dfs -rm -r ' + hdfs_path
        result_delete = subprocess.run(hdfs_command_delete, shell=True, capture_output=True, text=True)
        
        if result_delete.returncode == 0:
            print("Thư mục cũ đã được xóa thành công.")
        else:
            print("Có lỗi xảy ra khi xóa thư mục:", result_delete.stderr)
    else:
        print("Thư mục chưa tồn tại, sẽ được tạo mới.")

    # Lưu RDD dưới dạng tệp văn bản
    rdd.saveAsTextFile(hdfs_path)
    print("Dữ liệu đã được lưu thành công.")

####
####
####
#ham xu ly hdfs:
# Hàm để xóa các ký tự đặc biệt nhưng giữ lại các ký tự tiếng Việt có dấu
#lay list cac file chua tep data.txt
def list_hdfs_files(path):
    # Thực thi lệnh hdfs dfs -ls và lấy đầu ra
    output = subprocess.check_output(["hdfs", "dfs", "-ls", path]).decode("utf-8")
    
    # Phân tách đầu ra thành danh sách các dòng
    lines = output.splitlines()

    # Lọc ra tên của các tệp
    files = [line.split()[-1] for line in lines if not line.startswith("d")]

    return files[1:]

#chuyển rdd thành 1 đoạn text
def hdfs_to_text(file_path):
    # Đọc tệp văn bản vào một RDD
    rdd = sc.textFile(file_path)

    # Thu thập nội dung RDD thành một danh sách
    txt = " ".join(rdd.collect())
   
    return txt

#tiền xử lý
def preprocess_text(text, stop_words):
    # Chuyển văn bản thành chữ thường
    text = text.lower()
    text =text.replace('_', ' ')
    # Xóa các ký tự đặc biệt và số, chỉ giữ lại chữ cái và dấu cách
    text = re.sub(r'[^a-zA-ZÀ-ỹ\s]', '', text)
    # Xóa các từ dừng, bao gồm cả từ ghép
    for stop_word in stop_words:
        pattern = r'\b{}\b'.format(re.escape(stop_word))
        text = re.sub(pattern, '', text)
    
    # Xóa khoảng trắng thừa sau khi xóa từ dừng
    text = re.sub(r'\s+', ' ', text).strip()   
    
    # Ghép các từ lại thành chuỗi
    return text

#từ văn bản đưa ra list danh sách từ và vị trí từ đó xuất hiện trong đoạn văn
def text_to_word_info(text):
    # Chuyển văn bản thành chữ thường để không phân biệt chữ hoa và chữ thường
    text = text.lower()

    # Tách văn bản thành danh sách các từ
    words = text.split()

    # Tạo dictionary để lưu trữ thông tin về từ
    word_info = {}

    # Duyệt qua danh sách từ và cập nhật số lượng xuất hiện và vị trí
    for index, word in enumerate(words):
        if word in word_info:
            #word_info[word]['count'] += 1
            word_info[word]['positions'].append(index)
        else:
            word_info[word] = { 'positions': [index]}

    # Chuyển từ dictionary sang list các tuple (từ, số lượng, danh sách vị trí)
    word_list = [(word, info['positions']) for word, info in word_info.items()]

    return word_list






In [ ]:
#code tao rdd
# Đường dẫn đến thư mục trên HDFS
hdfs_path = newdata
link = 0
list_data = []
# Lấy danh sách các tệp trong thư mục trên HDFS
file_list = list_hdfs_files(hdfs_path)
len(file_list)

In [ ]:
data = [(i, (letter, '[]', '[]'))  for i, letter in enumerate(letters)] 
data_vertices= spark.sparkContext.parallelize(data)
vertices= data_vertices.collect()
global vertices,data_vertices

In [ ]:
list_data =[]
error_data=[]
i = 0
for file in file_list[:500]:
    i+=1
    print("Tạo cây",i,"từ file", file)
    try:
        text = hdfs_to_text(file)
        text = preprocess_text(text,stop_words)
        info_words = text_to_word_info(text)  
        path = int(os.path.splitext(os.path.basename(file))[0]) 
        for word,info in info_words:
            a = time.time()
            link = (path,info)
            check_word(word,link)
            print(word,time.time()-a)
        list_data.append(path)
    except:
        print('error in ',file)
        subprocess.run(["hdfs", "dfs", "-rm", file], check=True)
        error_data.append(path)
data_vertices= spark.sparkContext.parallelize(vertices)


In [ ]:
saveTreeData('verices_v1/vertices',data_vertices)


In [ ]:
local_folder_path = "/home/master/Downloads/Data/"
files = [f for f in os.listdir(local_folder_path) if os.path.isfile(os.path.join(local_folder_path, f))]
print(len(files))

In [ ]:
def read_files_from_folder(local_file_path):
    with open(local_file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return content
def saveTreeData(name, rdd):
    # Lấy ngày giờ hiện tại
    now = datetime.now()

    # Định dạng ngày giờ theo định dạng 'YYYY-MM-DD'
    formatted_time = now.strftime("%Y-%m-%d")

    # Đường dẫn HDFS để lưu đỉnh và cạnh
    link = "/save"
    hdfs_directory = link + '/' + str(formatted_time) + "/" + name +'/'
    hdfs_path = hdfs_directory+ 'vertices'

    # Câu lệnh HDFS để kiểm tra xem thư mục đã tồn tại chưa
    hdfs_command_check = 'hdfs dfs -test -d ' + hdfs_path

    # Sử dụng subprocess để kiểm tra sự tồn tại của thư mục
    result_check = subprocess.run(hdfs_command_check, shell=True, capture_output=True, text=True)

    # Nếu thư mục tồn tại (lệnh `hdfs dfs -test -d` trả về mã 0), thì xóa thư mục
    if result_check.returncode == 0:
        hdfs_command_delete = 'hdfs dfs -rm -r ' + hdfs_path
        result_delete = subprocess.run(hdfs_command_delete, shell=True, capture_output=True, text=True)
        
        if result_delete.returncode == 0:
            print("Thư mục cũ đã được xóa thành công.")
        else:
            print("Có lỗi xảy ra khi xóa thư mục:", result_delete.stderr)
    else:
        print("Thư mục chưa tồn tại, sẽ được tạo mới.")
        
    rdd.saveAsTextFile(hdfs_path)
    print("Dữ liệu đã được lưu thành công.")
    subprocess.run(["hdfs", "dfs", "-copyFromLocal", 'hdfs_success.json', hdfs_directory])
    print("đã đưa file hdfs_success.json lên hdfs")
    subprocess.run(["hdfs", "dfs", "-copyFromLocal", 'hdfs_error.json', hdfs_directory])
    # Lưu RDD dưới dạng tệp văn bản
    print("đã đưa file hdfs_error.json lên hdfs")

    


In [ ]:
success_data =[]
error_data=[]
i = 0
batch_size =1000
batches = [files[x:x+batch_size] for x in range(0, len(files), batch_size)]

for batch_index, batch in enumerate(batches):
    for file in batch:
        i += 1
        print(f"Tạo cây {i} từ file {file}")
        try:
            local_file_path = os.path.join(local_folder_path, file)
            text = read_files_from_folder(local_file_path)
            text = preprocess_text(text, stop_words)
            info_words = text_to_word_info(text)
            path = int(os.path.splitext(file)[0])
            for word, info in info_words:
                start_time = time.time()
                link = (path, info)
                check_word(word, link)
                print(word, time.time() - start_time)
            success_data.append(path)
        except Exception as e:
            print(f'Error in {file}: {e}')
            error_data.append(path)
    
    # Chuyển vertices thành RDD và lưu bằng saveTreeData
    data_vertices= spark.sparkContext.parallelize(vertices)
    print(len(success_data))
    with open('hdfs_success.json', "w") as f:
        json.dump(success_data, f, indent=4)
    print(len(error_data))
    with open('hdfs_error.json', "w") as f:
        json.dump(error_data, f, indent=4)
    batch_name = f"vertices_v{batch_index + 1}"
    saveTreeData(batch_name, data_vertices)
    
    

thời 0.20524072647094727
giá 0.2512216567993164
lạnh 0.09297776222229004
reuters 0.031597137451171875
nửa 0.0646674633026123
hệ 0.25504398345947266
hư 0.10019564628601074
hại 0.32143592834472656
trình 0.4271245002746582
đe 0.10900568962097168
dọa 0.057088613510131836
trận 0.18099069595336914
nghiêm 0.18539810180664062
trọng 0.4730658531188965
thành 0.4424912929534912
lập 0.20166277885437012
sản 0.27286767959594727
xuất 0.18967676162719727
dựa 0.2285480499267578
mẫu 0.14229202270507812
la 0.13304996490478516
hét 0.0641634464263916
brazy 0.07010459899902344
tình 0.248121976852417
thể 0.5155167579650879
sahard 0.09301447868347168
nhiễu 0.09712076187133789
hỏa 0.13783526420593262
syria 0.07967948913574219
đoán 0.1399695873260498
toán 0.1908719539642334
hạm 0.16224312782287598
tuyên 0.19245266914367676
ép 0.033745765686035156
chrome 0.09148073196411133
khăn 0.13890504837036133
đảo 0.4027104377746582
trụ 0.15617752075195312
biển 0.2035832405090332
đen 0.12251400947570801
crưmia 0.06854581832

toàn 0.33951306343078613
thuận 0.3815422058105469
lợi 0.13970398902893066
mặt 0.25248122215270996
khăn 0.11938905715942383
thách 0.13822221755981445
thức 0.19065451622009277
tác 0.23168683052062988
ninh 0.1368086338043213
trật 0.09949922561645508
hướng 0.18587994575500488
lãnh 0.1657581329345703
đạo 0.22287631034851074
xuyên 0.13231825828552246
sát 0.4776935577392578
đảng 0.12125945091247559
phối 0.3865509033203125
hợp 0.24194836616516113
chặt 0.11626648902893066
chẽ 0.11707472801208496
ban 0.2239542007446289
ngành 0.15000391006469727
địa 0.24984002113342285
trận 0.13799142837524414
tổ 0.22654390335083008
quốc 0.3440728187561035
đoàn 0.18013882637023926
ủng 0.057700395584106445
hộ 0.1535496711730957
giúp 0.1670687198638916
đỡ 0.07592225074768066
trung 0.2900731563568115
sức 0.19585180282592773
đồng 0.4247100353240967
nỗ 0.3263118267059326
phấn 0.09568953514099121
hoàn 0.12064743041992188
thành 0.44649672508239746
thắng 0.12175345420837402
nhiệm 0.18036699295043945
vụ 0.2645506858825683

hoạt 0.19109010696411133
lớp 0.06991863250732422
mộ 0.10279321670532227
liên 0.23016858100891113
hoan 0.09478092193603516
mạng 0.20856094360351562
xã 0.19503045082092285
hội 0.40381503105163574
dư 0.05648088455200195
luận 0.11533045768737793
xôn 0.05446672439575195
xao 0.03781294822692871
dục 0.08106589317321777
đào 0.16700363159179688
tổ 0.19380760192871094
chức 0.3707864284515381
xác 0.2183384895324707
minh 0.598799467086792
đề 0.1649782657623291
nghị 0.16893386840820312
địa 0.22229814529418945
phương 0.3346288204193115
xảy 0.15928125381469727
cáo 0.30701756477355957
diễn 0.2546658515930176
đàn 0.12978792190551758
kiến 0.16013693809509277
định 0.3362002372741699
quan 0.16666030883789062
xây 0.11639261245727539
dựng 0.3415861129760742
đối 0.3707468509674072
ngành 0.3695640563964844
sở 0.20093703269958496
hải 0.13573002815246582
dương 0.11730670928955078
chiều 0.3183298110961914
sáu 0.06871294975280762
nth 0.0351262092590332
chủ 0.2783682346343994
nhiệm 0.16861605644226074
c 0.07728719

công 0.6941933631896973
di 0.10417556762695312
sản 0.21329712867736816
văn 0.17297816276550293
hóa 0.1752176284790039
phi 0.10879731178283691
thể 0.41355419158935547
gia 0.3665618896484375
vây 0.039533138275146484
bắt 0.1421501636505127
cá 0.1347215175628662
sấu 0.041368961334228516
lẻn 0.03771328926086426
dân 0.33177781105041504
thả 0.08830070495605469
sông 0.0811614990234375
cải 0.10448694229125977
tổ 0.18797898292541504
sâu 0.11829161643981934
rộng 0.08654189109802246
cát 0.3591892719268799
thay 0.12201809883117676
trưởng 0.2001204490661621
david 0.0443267822265625
cameron 0.1453859806060791
thông 0.4320096969604492
chi 0.19326448440551758
tiết 0.16185355186462402
tức 0.15538716316223145
cập 0.1409289836883545
nhật 0.1316206455230713
gi 0.07615208625793457
âm 0.1972198486328125
nhạc 0.20558691024780273
đối 0.3453042507171631
hình 0.2491300106048584
hợp 0.26407957077026367
ngũ 0.2973313331604004
slip 0.050554513931274414
bx 0.04977726936340332
sở 0.18761682510375977
xây 0.11785030364

bốn 0.24437522888183594
dương 0.10657525062561035
ma 0.1165473461151123
túy 0.08319735527038574
phạt 0.15218305587768555
tiền 0.2620725631713867
tỷ 0.1819462776184082
huỳnh 0.08300161361694336
khoa 0.1773068904876709
ngoại 0.4507560729980469
đa 0.14597463607788086
tây 0.22495555877685547
mổ 0.10876107215881348
cứu 0.1927783489227295
đạn 0.12505841255187988
bắn 0.11637330055236816
xuyên 0.13202357292175293
b 0.042617082595825195
ngụ 0.07070493698120117
eek 0.014784574508666992
súng 0.06582450866699219
trúng 0.12124323844909668
chẫn 0.10637164115905762
vết 0.05872488021850586
sọ 0.030460834503173828
não 0.05735182762145996
hỏa 0.0620272159576416
khí 0.13485360145568848
vỡ 0.04909539222717285
thái 0.12178492546081543
tụ 0.07252812385559082
máu 0.11964964866638184
cứng 0.07322096824645996
dập 0.042987823486328125
huyết 0.13930654525756836
hiểm 0.1228630542755127
viên 0.35102105140686035
thước 0.13820505142211914
x 0.020618915557861328
mm 0.12027454376220703
phẫu 0.07193779945373535
thuật 0

...

nghiệp 0.22260737419128418
kinh 0.1841132640838623
xăng 0.04897809028625488
dầu 0.055108070373535156
biểu 0.14586925506591797
trây 0.07018542289733887
ỳ 0.0001544952392578125
chậm 0.11757349967956543
nộp 0.0673985481262207
tiền 0.20111489295959473
quỹ 0.03724861145019531
bình 0.22596406936645508
ổn 0.06881237030029297
tỷ 0.14971923828125
ty 0.1239161491394043
cổ 0.1220712661743164
thái 0.16917157173156738
quốc 0.5769796371459961
phòng 0.3052639961242676
trụ 0.12839746475219727
phùng 0.07491683959960938
khắc 0.11313223838806152
khoan 0.08641266822814941
đa 0.07439708709716797
kao 0.023673534393310547
dương 0.06513166427612305
đông 0.2973513603210449
hòa 0.14027905464172363
phú 0.13083481788635254
xóm 0.037479400634765625
tân 0.08717989921569824
thôn 0.10765457153320312
thủy 0.3214561939239502
xã 0.1924576759338379
huyện 0.22577810287475586
phong 0.1288301944732666
thuận 0.2675197124481201
gửi 0.09528517723083496
đôn 0.17636752128601074
đốc 0.11426401138305664
dư 0.05572390556335449
ngân

thẳng 0.10634708404541016
tra 0.14128708839416504
tạm 0.1000220775604248
dừng 0.0419459342956543
phát 0.37752461433410645
biểu 0.16249990463256836
đứng 0.12173819541931152
điện 0.647979736328125
kremlin 0.05509662628173828
nhấn 0.0939486026763916
chuyên 0.2164003849029541
sơ 0.10474371910095215
williams 0.010092020034790039
khăn 0.1350259780883789
đoán 0.10796761512756348
quy 0.16433191299438477
mô 0.14037084579467773
thức 0.19724607467651367
rộng 0.09539389610290527
kiếm 0.09015607833862305
bối 0.22357416152954102
cảnh 0.282912015914917
tương 0.2704200744628906
lai 0.1485764980316162
hiệu 0.17696595191955566
sức 0.15483593940734863
mù 0.0996389389038086
mờ 0.08927345275878906
tóm 0.04834413528442383
dõi 0.0956265926361084
diễn 0.19929981231689453
ukraine 0.03548550605773926
hướng 0.19221782684326172
trường 0.5409581661224365
khủng 0.11271977424621582
hoảng 0.07158780097961426
đề 0.19865083694458008
xuất 0.17131304740905762
thay 0.13332700729370117
hợp 0.2606375217437744
pháp 0.3081386

hoạt 0.19046711921691895
đời 0.3479647636413574
bổ 0.09859848022460938
sung 0.05973529815673828
chỉnh 0.08896183967590332
đối 0.27485227584838867
án 0.1531825065612793
góc 0.06354761123657227
độ 0.2609269618988037
hạng 0.24498414993286133
điền 0.3016479015350342
hai 0.3109414577484131
hành 0.46375393867492676
nhượng 0.05621767044067383
bảo 0.21625494956970215
vệ 0.11874222755432129
giá 0.18593764305114746
xử 0.06319904327392578
hòa 0.4017918109893799
giải 0.3073570728302002
ta 0.20715117454528809
dần 0.06505584716796875
đức 0.12510371208190918
phật 0.08913469314575195
đấy 0.10821104049682617
hữu 0.10675764083862305
nộp 0.04392552375793457
thuế 0.23383283615112305
đóng 0.15110230445861816
nghĩa 0.11016726493835449
vụ 0.33024072647094727
mục 0.18027424812316895
chương 0.19098877906799316
thúc 0.1542348861694336
đẩy 0.11542010307312012
lập 0.13216423988342285
minh 0.31003332138061523
bạch 0.08301353454589844
rộng 0.09031248092651367
giao 0.7655386924743652
tượng 0.2787437438964844
góp 0.0

chậm 0.12371253967285156
nín 0.03310513496398926
thở 0.10631060600280762
giây 0.08937954902648926
click 0.06769919395446777
chuột 0.11004376411437988
tắt 0.06285643577575684
kết 0.32844018936157227
nối 0.0671694278717041
wii 0.0030257701873779297
bluetooth 0.054520368576049805
cáo 0.24038243293762207
liên 0.19786858558654785
thực 0.18248438835144043
trạng 0.09694576263427734
rộng 0.09913444519042969
rãi 0.019732952117919922
tham 0.1661996841430664
bày 0.07509064674377441
tỏ 0.054999351501464844
tưởng 0.22949767112731934
khiêm 0.0568239688873291
tốn 0.13694143295288086
thừa 0.11949920654296875
dân 0.5547726154327393
vật 0.12493038177490234
lộn 0.11578702926635742
vụ 0.3176419734954834
tuần 0.1525120735168457
sức 0.15973877906799316
kinh 0.1914689540863037
ngạc 0.08049941062927246
nghiêm 0.17127490043640137
trọng 0.2004384994506836
nhấn 0.096527099609375
liệu 0.1506035327911377
thống 0.3072340488433838
kê 0.06658291816711426
lo 0.09873414039611816
ngại 0.09708356857299805
trải 0.09729695

truyền 0.2339620590209961
giáp 0.2216506004333496
đấu 0.15256214141845703
thọc 0.33449459075927734
sâu 0.09211206436157227
kịch 0.07021141052246094
đại 0.21603631973266602
bao 0.0879676342010498
rộng 0.058426856994628906
đòn 0.03594517707824707
thăm 0.09841465950012207
dò 0.0307772159576416
hồi 0.27927541732788086
hi 0.05070161819458008
vọng 0.08700704574584961
tài 0.2489461898803711
khối 0.08562278747558594
nato 0.08223724365234375
sức 0.16534423828125
mắc 0.15086793899536133
sai 0.11723446846008301
lầm 0.04199624061584473
tế 0.21948599815368652
bào 0.06428217887878418
quanh 0.3121047019958496
bút 0.056581735610961914
artemiser 0.04044628143310547
phủ 0.18221712112426758
tỉnh 0.29245471954345703
đơn 0.12183570861816406
lẻ 0.05092954635620117
dấu 0.32010793685913086
dịch 0.1580522060394287
cố 0.19145679473876953
vấn 0.08673524856567383
anton 0.33401060104370117
greenco 0.036725521087646484
tàu 0.12534427642822266
thừa 0.12735533714294434
bó 0.05909442901611328
thức 0.19863128662109375
